# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nyala,11.6771,24.6653,20.97,27,87,5.57,SD,1734488189
1,1,fort st. john,56.1015,-120.4910,-22.73,98,98,2.13,CA,1734488191
2,2,el haouaria,37.8730,11.0711,15.08,71,0,3.99,TN,1734488194
3,3,constantia,3.0685,114.9600,22.46,100,100,0.26,ID,1734488196
4,4,sinabang,1.1178,-52.8516,22.37,99,100,0.66,BR,1734488200


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
best_weather = city_data_df[
        (city_data_df["Max Temp"] >= 14) & 
        (city_data_df["Max Temp"] <= 38) & 
        (city_data_df["Cloudiness"] <= 80)
        ]

# Drop any rows with null values
best_weather = best_weather.dropna()

# Display sample data
best_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,el haouaria,37.8730,11.0711,15.08,71,0,3.99,TN,1734488194
8,8,hadibu,-25.1017,-46.9443,23.32,76,18,4.44,BR,1734488211
9,9,haiku-pauwela,12.6611,53.2410,24.19,66,56,8.73,YE,1734488213
11,11,waitangi,-35.4871,141.1917,24.78,22,0,4.54,AU,1734488220
12,12,jamestown,-30.5899,122.0904,29.59,24,0,5.97,AU,1734488221
27,27,gadzhiyevo,6.8649,0.2489,24.49,37,20,0.67,GH,1734488303
36,36,kerikeri,-4.9480,55.3191,26.78,77,63,3.51,SC,1734488337
38,38,arraial do cabo,32.6972,-118.7011,14.72,85,62,2.29,US,1734488351
40,40,caluula,29.4136,-104.7766,15.55,30,0,1.81,MX,1734488363
47,47,alatyr',17.8087,-4.3843,16.77,26,70,6.67,ML,1734488396


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,el haouaria,TN,37.8730,11.0711,71,
8,hadibu,BR,-25.1017,-46.9443,76,
9,haiku-pauwela,YE,12.6611,53.2410,66,
11,waitangi,AU,-35.4871,141.1917,22,
12,jamestown,AU,-30.5899,122.0904,24,
27,gadzhiyevo,GH,6.8649,0.2489,37,
36,kerikeri,SC,-4.9480,55.3191,77,
38,arraial do cabo,US,32.6972,-118.7011,85,
40,caluula,MX,29.4136,-104.7766,30,
47,alatyr',ML,17.8087,-4.3843,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 1000000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
el haouaria - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
waitangi - nearest hotel: Mallee Fowl Hotel Motel
jamestown - nearest hotel: No hotel found
gadzhiyevo - nearest hotel: No hotel found
kerikeri - nearest hotel: Banyan Tree Resort
arraial do cabo - nearest hotel: No hotel found
caluula - nearest hotel: Hotel de Coyame
alatyr' - nearest hotel: No hotel found
higashine - nearest hotel: No hotel found
rongelap - nearest hotel: No hotel found
kedainiai - nearest hotel: Bar Reef Resort
port alfred - nearest hotel: No hotel found
banjarmasin - nearest hotel: No hotel found
mopti - nearest hotel: Campements et chambres
tolanaro - nearest hotel: No hotel found
kalemie - nearest hotel: فندق بيتش إن
ujae - nearest hotel: No hotel found
kulia village - nearest hotel: No hotel found
pochep - nearest hotel: No hotel found
itoman - nearest hotel: La baie des anges
geraldton - nearest hotel: Hotel Ocea

,City,Country,Lat,Lng,Humidity,Hotel Name
2,el haouaria,TN,37.8730,11.0711,71,No hotel found
8,hadibu,BR,-25.1017,-46.9443,76,No hotel found
9,haiku-pauwela,YE,12.6611,53.2410,66,No hotel found
11,waitangi,AU,-35.4871,141.1917,22,Mallee Fowl Hotel Motel
12,jamestown,AU,-30.5899,122.0904,24,No hotel found
27,gadzhiyevo,GH,6.8649,0.2489,37,No hotel found
36,kerikeri,SC,-4.9480,55.3191,77,Banyan Tree Resort
38,arraial do cabo,US,32.6972,-118.7011,85,No hotel found
40,caluula,MX,29.4136,-104.7766,30,Hotel de Coyame
47,alatyr',ML,17.8087,-4.3843,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1,
    color="City",
    hover_cols=["Hotel Name", "Country"]  # Add hotel name and country to hover message
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)